# Clase 5 - XlsxWriter, estructura de un proyecto y ejemplos prácticos 💹📋

## XlsxWriter

Se instala con `pip install XlsxWriter` o `conda install XlsxWriter`

Es una libreria de python que sirve para crear hojas de calculo de formato "Excel" de forma automatizada y con una capacidad de configuración enorme. Con XlsxWriter vamos a poder crear archivos excel de múltiples hojas a partir de muchos dataframes. Además nos va a dar la posibilidad de crear archivos excel usando casi todo lo que excel nos ofrece. Vamos a poder escribir formulas de excel, dar formato a las celdas, automatizar la creación de gráficos de excel o hasta escribir macros programáticamente.
Muchas veces por más automatizados y complejos que sean nuestros procesos es necesario compartirlos en forma de hojas de cálculo simplemente por la facilidad y familiaridad que tiene esa herramienta. Cuándo queremos exportar proyectos complejos a un excel es usual que un excel de una sola hoja con una tabla plana no sea suficiente satisfacer el objetivo del proyecto, para eso podemos usar esta libreria para sacarle más provecho al formato de excel.

¡Vean la documentación acá!: [Documentación XlsxWriter](https://xlsxwriter.readthedocs.io/)

Veamos ejemplos:

In [2]:
import xlsxwriter

workbook = xlsxwriter.Workbook('hello.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'Hello world')

workbook.close()

En este ejemplo ven como crea un nuevo archivo de excel y explícitamente en escribe algo en la celda 'A1'.
Quiero que noten como debe usar el método `.close()` para dejar de trbajar con el archivo. Es super importante no olvidarse de cerrar los archivos porque al igual que cuando uno usa un excel, ninguna otra persona ni proceso puede modificarlo mientras esté abierto. No cerrarlo va a causar que no sea seguro que los procesos que quisimos plasmar en el excel se guarden y tambien puede generar que su programa no pueda terminar correctamente o que directamente se quede abierto imposibilitando usar el archivo.

Por esa razón, para no olvidarse de cerrar el archivo, recomiendo mucho esta otra forma de escribir el mismo código:

In [3]:
with xlsxwriter.Workbook('hello.xlsx') as workbook:
    worksheet = workbook.add_worksheet()
    worksheet.write('A1', 'Hello world')


De esa forma abrimos o creamos el documento con `with` y una vez que salimos de ese bloque de código, el archivo se cierra de manera segura. Tengan en cuenta este patrón cada vez que en Python encuentren este tipo de objetos que deban ser cerrados por un método como `.close()` ya que posiblemente funcionen con `with` y de esa forma dejemos un código más claro y seguro.

### El objeto `Workbook`, `Worksheet` y otros
Fijense como para usar esta librería debemos usar los objetos que nos provee. Para hacer la hoja de cálculo más simple debemos llamar por lo menos a dos objetos. Al objeto `Workbook` que va a representar en el Python a nuestro archivo de excel y al objeto `WorkSheet` que representa a una hoja del excel.

Noten esto: primero llamamos a Workbook con el nombre del archivo que queremos modificar o leer (`workbook = xlsxwriter.Workbook('hello.xlsx')` o de la otra forma equivalente `with xlsxwriter.Workbook('hello.xlsx') as workbook:`). Luego es a partir del objeto que llamamos workbook que podemos traer nuevos objetos que como `Worksheet` a través de los métodos `.add_????()`que van a representar otras funcionalidades del excel. En el ejemplo de arriba creo solo una hoja a la que le escribe algo.

En el siguiente ejemplo escribimos dos hojas y le ponemos formato a las celdas con el objeto `Format`:

In [6]:
with xlsxwriter.Workbook('hello.xlsx') as libro2:
    hoja1 = libro2.add_worksheet()
    hoja2 = libro2.add_worksheet()
    formato_celda_resaltada = libro2.add_format({'bold':True,"bg_color":"yellow", 'font_color': 'red'})
    hoja1.write('A1', 'Hola mundo', formato_celda_resaltada)
    hoja2.write(0,0,'Chau')

Tengamos en cuenta que de la misma manera que puse un texto en una celda, tambien podemos escribir fórmulas de excel. Y asi como puedo agregar hojas al proyecto también puedo agregar gráficos si quisiese.

### Pandas y XlsxWriter 🐼 📊

Esta libreria funciona muy bien integrada con Pandas, para ello debemos aprovechar el objeto `ExcelWriter` que es la herramienta de pandas para integrarse muchas librerias para exportar los datos. Van a ver que se una muy parecido a los objetos que estabamos viendo antes, en este caso tenemos que indicarle a pandas que estamos usando como motor de exportación a __xlsxwriter__

In [172]:
import pandas as pd

df1 = pd.DataFrame({'uno':[1,2,3], 'dos':[3,2,1]})
df2 = pd.DataFrame({'hola':['a','b','c'], 'chau':[9,8,7]})

with pd.ExcelWriter('pandas.xlsx', engine='xlsxwriter') as writer:
    # Escribimos los datos en dos hojas distintas del excel
    df1.to_excel(writer,sheet_name='hoja1',index=False)
    df2.to_excel(writer,sheet_name='hoja2',index=False)
    
    #Usamos métodos de xlsxwriter para editarlo
    # Primero recuperamos los objetos que modelan al excel (libro, hojas, formato, etc)
    workbook: xlsxwriter.Workbook = writer.book
    hoja1:xlsxwriter.Workbook.worksheet_class = writer.sheets["hoja1"]
    hoja2:xlsxwriter.Workbook.worksheet_class = writer.sheets["hoja2"]
    
    formato = workbook.add_format({'bold':True,"bg_color":"yellow", 'font_color': 'red'})
    
    # pintemos con este formato a la primer columna de cada hoja
    for df, hoja in [(df1, hoja1),(df2, hoja2)]:
        max_row, max_col = df.shape
        for row in range(max_row):
            hoja.write(row+1,0,df.iloc[row,0],formato)
    
    
    

Si bien en este ejemplo no lo uso por la simplicidad que tiene, una de los cosas más útiles del objeto de formato es poder cambiar el formato de los numneros. Por ejemplo lo siguiente es un formato válido: `money = workbook.add_format({'num_format': '$#,##0'})`

## Ejercicio en clase

El objetivo es leer y limpiar un excel de complejos exportadores para luego exportarlo en otro excel en el que en cada hoja tenga un complejo exportador distinto. Además podemos usar formato condicional en las celdas para diferenciar las variaciones positivas de las negativas.

In [146]:
var_complejos = pd.read_excel("complexp_variacion_2019_2022.xls", header=3).dropna()

var_complejos.columns = [str(columna).replace('*','') for columna in var_complejos.columns]

var_complejos_etiquetas = var_complejos.iloc[:,0]

var_complejos = var_complejos.iloc[:,1:].astype(float)

var_complejos_etiquetas = pd.DataFrame({
    "item":var_complejos_etiquetas, 
    "nivel":var_complejos_etiquetas.apply(lambda item: (len(item) - len(item.lstrip())) // 2)})

maximo_nivel = var_complejos_etiquetas.nivel.max()
for nivel in range(maximo_nivel+1):
    var_complejos_etiquetas[f'nivel{nivel}'] = var_complejos_etiquetas.apply(
        lambda fila: fila['item'] if fila['nivel']==nivel else pd.NA, axis=1)
for nivel in range(maximo_nivel):
    var_complejos_etiquetas[f'nivel{nivel}'] = var_complejos_etiquetas[f'nivel{nivel}'].ffill()
    var_complejos_etiquetas[var_complejos_etiquetas['nivel'] == nivel] = var_complejos_etiquetas.query('nivel == @nivel').ffill(axis=1)
var_complejos_etiquetas = var_complejos_etiquetas.drop('item',axis=1)
var_complejos = pd.concat([var_complejos_etiquetas,var_complejos],axis=1)

C:\Users\lucag\AppData\Local\Temp\ipykernel_11116\2526349808.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  var_complejos_etiquetas[var_complejos_etiquetas['nivel'] == nivel] = var_complejos_etiquetas.query('nivel == @nivel').ffill(axis=1)


In [154]:
var_complejos_agrupada_por_n0 = var_complejos.groupby("nivel0")
